In [17]:
from MCTS import MCTS
from sogo.SogoGame import SogoGame, display as display_board
import numpy as np
from NeuralNet import NeuralNet
from Game import Game

g = SogoGame(4)

class Config(object):
    def __init__(self):    
      self.max_moves = 512  # for chess and shogi, 722 for Go.
      self.numMCTSSims = 1000

      # Root prior exploration noise.
      self.root_dirichlet_alpha = 0.3  # for chess, 0.03 for Go and 0.15 for shogi.
      self.root_exploration_fraction = 0.0

      # UCB formula
      self.pb_c_base = 19652
      self.pb_c_init = 1.25


class NN(NeuralNet):
  def __init__(self,game:Game):
    self.game = game
  def predict(self, board):
    return np.ones(self.game.getActionSize())/self.game.getActionSize(), 0

config = Config()

nn = NN(g)
mcts1 = MCTS(g, nn, config)
player = 1
def mcts_player(x):
    pi, root = mcts1.get_action_prob(g.getCanonicalForm(x,player))
    return np.argmax(pi), root

In [3]:
import time

class Timer:    
    def __enter__(self):
        self.start = time.clock()
        return self

    def __exit__(self, *args):
        self.end = time.clock()
        self.interval = self.end - self.start

In [18]:
def setup_board(plays): 
    board = g.getInitBoard()
    player = 1    
    for play in plays:
        board, player = g.getNextState(board, player,play)
    display_board(board)
    return board, player

In [19]:
def test_mcts(plays, expected):
    board, player = setup_board(plays)    
    with Timer() as t:
        play, root = mcts_player(board)    
    new_board, new_player = g.getNextState(board, player,play)
    display_board(new_board)
    print(f"MCTS made {'correct' if play == expected else 'incorrect' } play in  {t.interval:0.3f} sec")
    return root

### State with easy win

In [8]:
test_mcts([0,8,0,8,0,8],0);


z3+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- - - - |
z3+--------+
   0 1 2 3 
z2+--------+
3 |- - - - |
2 |X - - - |
1 |- - - - |
0 |O - - - |
z2+--------+
   0 1 2 3 
z1+--------+
3 |- - - - |
2 |X - - - |
1 |- - - - |
0 |O - - - |
z1+--------+
   0 1 2 3 
z0+--------+
3 |- - - - |
2 |X - - - |
1 |- - - - |
0 |O - - - |
z0+--------+
   0 1 2 3 
--
z3+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |O - - - |
z3+--------+
   0 1 2 3 
z2+--------+
3 |- - - - |
2 |X - - - |
1 |- - - - |
0 |O - - - |
z2+--------+
   0 1 2 3 
z1+--------+
3 |- - - - |
2 |X - - - |
1 |- - - - |
0 |O - - - |
z1+--------+
   0 1 2 3 
z0+--------+
3 |- - - - |
2 |X - - - |
1 |- - - - |
0 |O - - - |
z0+--------+
   0 1 2 3 
--
MCTS made correct play in  3.906 sec


### States that requires defense against 1 step win

In [ ]:
test_mcts([0,8,0,8,0],0);


In [ ]:
b,p = setup_board([0,7,3,11,5,15,13,1,0,9]); # from 40k runs or so
g.getGameEnded(b,p)


### State requires defense against 2 step win as player 2

In [26]:
config.numMCTSSims = 1000
test_mcts([0,7,3,11,5,15,13],1);


z3+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- - - - |
z3+--------+
   0 1 2 3 
z2+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- - - - |
z2+--------+
   0 1 2 3 
z1+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- - - - |
z1+--------+
   0 1 2 3 
z0+--------+
3 |- O - X |
2 |- - - X |
1 |- O - X |
0 |O - - O |
z0+--------+
   0 1 2 3 
--
z3+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- - - - |
z3+--------+
   0 1 2 3 
z2+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- - - - |
z2+--------+
   0 1 2 3 
z1+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- - - - |
z1+--------+
   0 1 2 3 
z0+--------+
3 |- O - X |
2 |X - - X |
1 |- O - X |
0 |O - - O |
z0+--------+
   0 1 2 3 
--
MCTS made incorrect play in  2.267 sec


### State requires defense against 2 step win as player 1

In [28]:
config.numMCTSSims = 10000
root = test_mcts([12,0,7,3,11,5,15,13],1);


z3+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- - - - |
z3+--------+
   0 1 2 3 
z2+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- - - - |
z2+--------+
   0 1 2 3 
z1+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- - - - |
z1+--------+
   0 1 2 3 
z0+--------+
3 |O X - O |
2 |- - - O |
1 |- X - O |
0 |X - - X |
z0+--------+
   0 1 2 3 
--
z3+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- - - - |
z3+--------+
   0 1 2 3 
z2+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- - - - |
z2+--------+
   0 1 2 3 
z1+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |O - - - |
z1+--------+
   0 1 2 3 
z0+--------+
3 |O X - O |
2 |- - - O |
1 |- X - O |
0 |X - - X |
z0+--------+
   0 1 2 3 
--
MCTS made incorrect play in  25.216 sec


### State with 2 step win as player 1

In [30]:
config.numMCTSSims = 1000
root = test_mcts([0,7,3,11,5,15,13,12],1);

z3+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- - - - |
z3+--------+
   0 1 2 3 
z2+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- - - - |
z2+--------+
   0 1 2 3 
z1+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- - - - |
z1+--------+
   0 1 2 3 
z0+--------+
3 |X O - X |
2 |- - - X |
1 |- O - X |
0 |O - - O |
z0+--------+
   0 1 2 3 
--
z3+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- - - - |
z3+--------+
   0 1 2 3 
z2+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- - - - |
z2+--------+
   0 1 2 3 
z1+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- - - - |
z1+--------+
   0 1 2 3 
z0+--------+
3 |X O - X |
2 |O - - X |
1 |- O - X |
0 |O - - O |
z0+--------+
   0 1 2 3 
--
MCTS made incorrect play in  2.579 sec


### State with 2 step win as player 2

In [33]:
config.numMCTSSims = 1000
root = test_mcts([14,0,7,3,11,5,15,13,12],1);

z3+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- - - - |
z3+--------+
   0 1 2 3 
z2+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- - - - |
z2+--------+
   0 1 2 3 
z1+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- - - - |
z1+--------+
   0 1 2 3 
z0+--------+
3 |O X O O |
2 |- - - O |
1 |- X - O |
0 |X - - X |
z0+--------+
   0 1 2 3 
--
z3+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- - - - |
z3+--------+
   0 1 2 3 
z2+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- - - - |
z2+--------+
   0 1 2 3 
z1+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- - - - |
z1+--------+
   0 1 2 3 
z0+--------+
3 |O X O O |
2 |X - - O |
1 |- X - O |
0 |X - - X |
z0+--------+
   0 1 2 3 
--
MCTS made incorrect play in  2.509 sec


In [36]:
root.print(2)
root.children[1].print(2)
root.children[1].children[0].print(2)

 -> v:0.0 n:1000 p:0.0 tp:-1
  0 -> v:0.0 n:62 p:0.062 tp:1
  1 -> v:0.0 n:62 p:0.062 tp:1
  2 -> v:0.0 n:62 p:0.062 tp:1
  3 -> v:0.0 n:62 p:0.062 tp:1
  4 -> v:0.0 n:62 p:0.062 tp:1
  5 -> v:0.0 n:62 p:0.062 tp:1
  6 -> v:0.0 n:62 p:0.062 tp:1
  7 -> v:0.0 n:62 p:0.062 tp:1
  8 -> v:0.0 n:63 p:0.062 tp:1
  9 -> v:0.0 n:63 p:0.062 tp:1
  10 -> v:0.0 n:63 p:0.062 tp:1
  11 -> v:0.0 n:63 p:0.062 tp:1
  12 -> v:0.0 n:63 p:0.062 tp:1
  13 -> v:0.0 n:63 p:0.062 tp:1
  14 -> v:0.0 n:63 p:0.062 tp:1
  15 -> v:0.0 n:63 p:0.062 tp:1
 -> v:0.0 n:62 p:0.062 tp:1
  0 -> v:0.0 n:3 p:0.062 tp:-1
  1 -> v:0.0 n:3 p:0.062 tp:-1
  2 -> v:0.0 n:3 p:0.062 tp:-1
  3 -> v:0.0 n:4 p:0.062 tp:-1
  4 -> v:0.0 n:4 p:0.062 tp:-1
  5 -> v:0.0 n:4 p:0.062 tp:-1
  6 -> v:0.0 n:4 p:0.062 tp:-1
  7 -> v:0.0 n:4 p:0.062 tp:-1
  8 -> v:0.0 n:4 p:0.062 tp:-1
  9 -> v:0.0 n:4 p:0.062 tp:-1
  10 -> v:0.0 n:4 p:0.062 tp:-1
  11 -> v:0.0 n:4 p:0.062 tp:-1
  12 -> v:0.0 n:4 p:0.062 tp:-1
  13 -> v:0.0 n:4 p:0.062 tp:-1
  1

In [10]:
import uuid
from IPython.display import display_javascript, display_html, display
import json

class RenderJSON(object):
    def __init__(self, json_data):
        if isinstance(json_data, dict):
            self.json_str = json.dumps(json_data)
        else:
            self.json_str = json
        self.uuid = str(uuid.uuid4())
        
    def _ipython_display_(self):
        display_html('<div id="{}" style="height: 600px; width:100%;"></div>'.format(self.uuid),
            raw=True
        )
        display_javascript("""
        require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {
          document.getElementById('%s').appendChild(renderjson(%s))
        });
        """ % (self.uuid, self.json_str), raw=True)